<a href="https://colab.research.google.com/github/MatheusPiassiC/redes_neurais/blob/main/neuronio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estudo de implementação de neurônio simples para uma rede neural
Baseado no notebook do professor Denilson Alves Pereira, junto de seus conteúdos gravados.

In [97]:
import numpy as np
import matplotlib.pyplot as plt

##Enunciado
O problema consiste em classificar um conjunto de dados hipotéticos. Cada instância é composta por 4 atributos (0 ou 1), sendo que a saída deve ser um 0 ou 1 (classificaçãso binária), e é determinada pelo valor do primeiro atributo da instância.

Ou seja, se a instância for [1,0,0,1], a saída será 1

Vale lembrar que neste código, as instâncias são representadas pelas colunas, e os atributos pelas linhas, como mostrado no trecho de código abaixo.

In [98]:
X_train = np.array([[0, 1, 0, 1, 1, 0],   #como dito anteriormente, são 6 instâncias de 4 atributos.
                    [0, 1, 1, 0, 0, 1],
                    [0, 0, 0, 1, 0, 1],
                    [1, 0, 0, 0, 1, 1]])

y_train = np.array([0, 1, 0, 1, 1, 0])    #saída esperada do conjunto de treino



In [99]:
X_test = np.array([[1, 0, 1], #conjunto de teste
                   [1, 1, 1],
                   [0, 1, 1],
                   [1, 0, 1]])

y_test = np.array([[1, 0, 1]]) #saidas esperadas do conjunto de teste

In [100]:
n = X_train.shape[0] #numero de atributos (shape retorna um array com o numero de linhas e colunas)
m = X_train.shape[1] #numero de exemplos de treino

print("número de atributos: n=" + str(n))
print("número de exemplos de treino: m=" + str(m))
print("forma de X_train: "+ str(X_train.shape))
print("forma de y_train: "+ str(y_train.shape))
print("forma de X_test: "+ str(X_test.shape))
print("forma de y_test: "+ str(y_test.shape))

número de atributos: n=4
número de exemplos de treino: m=6
forma de X_train: (4, 6)
forma de y_train: (6,)
forma de X_test: (4, 3)
forma de y_test: (1, 3)


##Arquitetura
A partir daqui, será implementado um classificador binário usando regressão logistica para resolver o problema.

Calculo de z de uma instância x:

$$z^{(i)} = w^T x^{(i)} + b$$

Saída do neurônio usando função de ativação sigmoide:

$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})$$

Função de perda da regressão logistica:

$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})$$

OBS: A função de custo consiste no somatório das funções de perda.

###Inicialização dos parâmetros
Seguindo o exemplo utilizado, o próximo passo é iniciar os parametros *b* (bias, viés) e *w* (pesos para cada atributo). O tamanho do vetor *w* é igual ao número de atributos de uma entrada (neste caso, 4), pois precisamos de atribuir um peso para cada atributo, enquanto *b* é um escalar.

In [101]:
def initializeParameters(dim): #recebe o dim para saber o tamanho do vetor w
  w = np.zeros((dim,1)) #4 LINHAS e 1 coluna, para seguir o padrâo do código
  b = 0.0 #inicializa o bias
  return w, b

In [102]:
dim = n
w,b = initializeParameters(dim)
print("w = "+ str(w))
print("b = "+ str(b))

w = [[0.]
 [0.]
 [0.]
 [0.]]
b = 0.0


###Função sigmóide
Este trecho computa a função sigóide a partir de um *z*:

$sigmoid(z) = \frac{1}{1 + e^{-(z)}}$

In [103]:
def sigmoid(z):
  s = 1/(1 + np.exp(-z))
  return s

In [104]:
print ("sigmoid([0, 1, 2, 8]) = " + str(sigmoid(np.array([0,1,2,8]))))

sigmoid([0, 1, 2, 8]) = [0.5        0.73105858 0.88079708 0.99966465]


###Propagação para frente (feed foward)
Executa a propagaçãom para frente através do calculo da função sigmoide para um determinado *z* ( considere que $z = w^T X + b$ ).
Através no *numpy*, este cálculo pode ser feito de forma paralelizada para vários valores, por isso, usaremos um vetor X e calcularemos a sigmoide para cada *x* (lembrando que x é uma instância) em *X* (que é uma matriz)

Deste modo, o calculo de $z$ seria algo como:

$z = x^{(1)}w^{(1)} + x^{(2)}w^{(2)} + ... + b$

Sendo X = [[0, 1, 0, 1, 1, 0],  
          [0, 1, 1, 0, 0, 1],
          [0, 0, 0, 1, 0, 1],
          [1, 0, 0, 0, 1, 1]]

E sendo W = [[1],
            [2],
            [3],
            [4]]

Nós queremos múltiplicar cada linha de uma coluna de X (que corresponde a uma instância/entrada) por cada linha de W. Para mantermos a propriedade que permite a multiplicação de matrizes (o número de colunas da primeira matriz deve ser igual ao número de linhas da segunda matriz), precisamos de transpor W. Segue um exemplo para a primeira entrada (ou seja, primeira coluna) de X.

Daí, teriamos algo parecido como:

$z = (0*1) + (0*2) + (0*3) + (1*4) + b$

Essa fórmula para o cálculo de $z$ é usada como parêmtro para a sigmoide, que retorna uma lista de resoltados para cada coluna de X.

 $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$

In [105]:
def feed_forward(w,b,X):
  A = sigmoid(np.dot(w.T, X)+b) #np.dot realiza multiplicações de vetores e matrizes
  return A

In [106]:
#teste 1
w, b, X = np.array([[0.5],[2.]]), 2., np.array([[1.,-2.,1.],[4.,3.,-2.3]])
A = feed_forward(w, b, X)
print(A)

[[0.99997246 0.99908895 0.10909682]]


In [107]:
#teste 2
w, b = initializeParameters(dim)
A = feed_forward(w, b, X_train)
print(A)

[[0.5 0.5 0.5 0.5 0.5 0.5]]


###Propagação para trás (backpropagation)
Computa a função de custo, ou seja, o a média das funções de perda para cada entrada. Também calcula as derivadas parciais de $w$ e $b$.

In [108]:
def backpropagation(A,X,Y): #"A" é o conjunto de saidas da sigmoide
  m = X.shape[1] #número de colunas de X, ou seja, o número de exemplos
  cost = -(np.sum(Y*np.log(A)+(1-Y)*np.log(1-A)))/m #média dos somatório das funções de perda, ou seja, a função de custo
  dw = (np.dot(X,(A-Y).T))/m #calcula dw
  db = (np.sum(A-Y))/m       #calcula db
  cost = np.squeeze(cost) #remove eixos desmecessários
  grads = {"dw": dw,
           "db": db}
  return grads, cost

In [109]:
w, b, X, Y = np.array([[0.5],[2.]]), 2., np.array([[1.,-2.,1.],[4.,3.,-2.3]]), np.array([[1,0,1]])
A = feed_forward(w, b, X)
grads, cost = backpropagation(A, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[-0.9630362]
 [ 1.682078 ]]
db = 0.03605274477003254
cost = 3.072152841901268


##Modelo de treino
Este trecho executa o algoritmo de descida de gradiente para otimizar os valores dos parâmetros $w$ e $b$. O objetivo é aprender os melhores valores para as variáveis $w$ e $b$ de modo a minimizar a função de custo.

Para um parâmetro $\theta$, a regra de atualização é $\theta = \theta - \alpha \text{ } d\theta$, onde $\alpha$ é a taxa de aprendizagem.

In [110]:
def train(X, Y, epochs = 1000, alpha = 0.005, print_cost = False):
  costs = []
  n = X.shape[0] #número de atributos de cada instância
  w,b = initializeParameters(n) #inicializa w e b

  for i in range(epochs):
    A = feed_forward(w,b,X) #calcula as saidas com base na função sigmoide
    grads, cost = backpropagation(A,X,Y) #faz o backpropagation
    dw = grads["dw"]  #guarda dw
    db = grads["db"]  #guarda db
    w = w - alpha * dw #atualiza os pesos w com base em dw e na taxa de aprendizado alpha
    b = b - alpha * db #atualiza o bias com base em db e na taxa de aprendizado
    if i % 100 == 0:
      costs.append(cost)
    if print_cost and i % 100 == 0:
      print ("Custo na iteração %i: %f" %(i, cost))
  params = {"w": w,
            "b": b}
  return params, costs




In [111]:
#testando
X, Y = np.array([[1.,-2.,1.],[4.,3.,-2.3]]), np.array([[1,0,1]])

params, costs = train(X, Y, epochs = 100, alpha = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))

w = [[ 0.47673328]
 [-0.05236593]]
b = 0.14693555740530975


##Predição de novos valores
Agora que já treinamos o nosso neurônio, podemos usar os pesos e o bias obtido para tentar prever novos valores.

Basicamente, executamos o feed forward e em seguida convertemos os valores para o padrão de saída:

  Se a saida for <= 0.5:  0
  
  Se a saída for > 0.5: 1

In [112]:
def predict(w, b, X): #prediz um conjunto de saídas para um determinado X, com base nos pesos e no bias
  m = X.shape[1] #número de colunas de X, ou seja, número de instâncias
  Y_pred = np.zeros((1,m))
  w = w.reshape(X.shape[0], 1) #transforma w em um vetor coluna
  A = feed_forward(w, b, X) #faz o fedd forward
  for i in range(A.shape[1]):
    Y_pred[0,i] = 1 if A[0,i] > 0.5 else 0 #transforma as saídas em 0 ou 1
  return Y_pred


In [113]:
w = np.array([[1.24076588],[0.75978603]])
b = -0.18
X = np.array([[0,0,0],[0,1,1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[0. 1. 1.]]


##Avaliação do modelo
A avaliação proposta é feita da seguinte forma:
1. Usa os dados de treinamento para gerar o *modelo de aprendizagem*, para aprender $w$ e $b$.
2. Calcula a acurácia obtida pelo modelo de treino.
3. Obtém a predição a partir dos dados de teste, com os parâmetros aprendidos.
4. Calcula a acurácia obtida nos dados de teste.

O esperado é que o custo diminua com o passar das iterações durante o aprendizado. Como o modelo é bastante simples, com apenas uma *epoch* já é possível obter acurácia de 100%.

In [114]:
print("treinando...")
params, costs = train(X_train, y_train, epochs = 1000, alpha = 0.005, print_cost = True)
print("treinado!")
w = params["w"]
b = params["b"]


print("Acurácia do treino: ")
y_pred = predict(w, b, X_train)
print(100 - np.mean(np.abs(y_pred - y_train)) * 100)

print("testando...")
y_pred = predict(w, b, X_test)
print(100 - np.mean(np.abs(y_pred - y_test)) * 100)

treinando...
Custo na iteração 0: 0.693147
Custo na iteração 100: 0.656550
Custo na iteração 200: 0.622944
Custo na iteração 300: 0.592033
Custo na iteração 400: 0.563555
Custo na iteração 500: 0.537278
Custo na iteração 600: 0.512996
Custo na iteração 700: 0.490522
Custo na iteração 800: 0.469690
Custo na iteração 900: 0.450349
treinado!
Acurácia do treino: 
100.0
testando...
100.0
